In [1]:
import numpy as np
import pandas as pd
from scipy.io import mmread
import os
import glob
import anndata
import zarr

- merge marker genes (types ABC)
- feed into NN design

In [2]:
pth = '/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P38_nr_1_a/filtered_feature_bc_matrix'
!ls $pth

barcodes.tsv.gz  counts.h5ad  features.tsv.gz  matrix.mtx


In [3]:
output = os.path.join(pth, 'counts.h5ad')
print(output)

/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P38_nr_1_a/filtered_feature_bc_matrix/counts.h5ad


In [4]:
%%time 
# 43s
fc = os.path.join(pth, 'barcodes.tsv.gz')
fg = os.path.join(pth, 'features.tsv.gz')
fm = os.path.join(pth, 'matrix.mtx')

cells = pd.read_csv(fc, sep='\t', header=None)[0].values
genes = pd.read_csv(fg, sep='\t', header=None).rename(columns={0: 'id', 1: 'name'})[['id', 'name']]
mtx = mmread(fm)
print(cells.shape, genes.shape, mtx.shape)

(10357,) (53801, 2) (53801, 10357)
CPU times: user 1min 26s, sys: 744 ms, total: 1min 27s
Wall time: 2min 49s


In [5]:
adata = anndata.AnnData(
    X=mtx.T.tocsc(), 
    obs=pd.DataFrame(index=cells), 
    var=genes,
)
adata

/home/fangming/anaconda3/envs/routine/lib/python3.10/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 10357 × 53801
    var: 'id', 'name'

In [6]:
%%time
adata.write(output)

/home/fangming/anaconda3/envs/routine/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'name' as categorical


CPU times: user 1.23 s, sys: 280 ms, total: 1.51 s
Wall time: 4.38 s


In [7]:
adata2 = anndata.read(output)
adata2

AnnData object with n_obs × n_vars = 10357 × 53801
    var: 'id', 'name'